In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
#minha funcao para mostrar as imagens
def imshow(im,big=False):
    if big:
        plt.figure(figsize=(20,20))
    else:
        plt.figure()
    if len(im.shape)<3:
        plt.imshow(im, cmap='gray')#As cores ficam zoadas no Jupyter se deixar sem esse parâmetro
    else:
        plt.imshow(im[:,:,::-1]) 
    plt.axis('off')
    plt.show()

***
# Contornos

In [ ]:
im = cv2.imread('imgs/im.jpg')
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
imshow(im_gray)

cv2.findContours
### mode:
RETR_EXTERNAL 	
retrieves only the extreme outer contours. It sets hierarchy[i][2]=hierarchy[i][3]=-1 for all the contours.

RETR_LIST 	
retrieves all of the contours without establishing any hierarchical relationships.

RETR_CCOMP 	
retrieves all of the contours and organizes them into a two-level hierarchy. At the top level, there are external boundaries of the components. At the second level, there are boundaries of the holes. If there is another contour inside a hole of a connected component, it is still put at the top level.

RETR_TREE 	
retrieves all of the contours and reconstructs a full hierarchy of nested contours.

RETR_FLOODFILL 	

### method:
CHAIN_APPROX_NONE 	
stores absolutely all the contour points. That is, any 2 subsequent points (x1,y1) and (x2,y2) of the contour will be either horizontal, vertical or diagonal neighbors, that is, max(abs(x1-x2),abs(y2-y1))==1.

CHAIN_APPROX_SIMPLE 	
compresses horizontal, vertical, and diagonal segments and leaves only their end points. For example, an up-right rectangular contour is encoded with 4 points.

CHAIN_APPROX_TC89_L1 	
applies one of the flavors of the Teh-Chin chain approximation algorithm [168]

CHAIN_APPROX_TC89_KCOS 	
applies one of the flavors of the Teh-Chin chain approximation algorithm [168]

In [ ]:
ret,thresh = cv2.threshold(im_gray,127,255,0)
con,h = cv2.findContours(thresh,mode=cv2.RETR_TREE,method = cv2.CHAIN_APPROX_SIMPLE)
len(con)

### Isso aqui não funcionou direito - tentativa de suavizar os contornos

In [ ]:
from scipy.interpolate import splprep, splev

In [ ]:
def suaviza_contorno (contorno):
    x,y = contorno.T
    
    # Convert from numpy arrays to normal arrays
    x = x.tolist()[0]
    y = y.tolist()[0]
    
    # https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.interpolate.splprep.html
    tck, u = splprep([x,y], u=None, s=1.0, per=1)
    
    # https://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.linspace.html
    u_new = np.linspace(u.min(), u.max(), 1000)
    
    # https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.interpolate.splev.html
    x_new, y_new = splev(u_new, tck, der=0)
    
    # Convert it back to numpy format for opencv to be able to display it
    res_array = [[[int(i[0]), int(i[1])]] for i in zip(x_new,y_new)]
    return np.asarray(res_array, dtype=np.int32)

In [ ]:
con_suavizado = []
con_suavizado.append(suaviza_contorno(con[0]))
con_suavizado.append(suaviza_contorno(con[1]))

### Testando o método de interpolação splprep do scipy

In [ ]:
from scipy.interpolate import splprep, splev

In [ ]:
phi = np.linspace(0, 2.*np.pi, 40)
r = 0.5 + np.cos(phi)
x, y = r * np.cos(phi), r * np.sin(phi)

In [ ]:
tck, u = splprep([x, y], s=0, per=1)
u_new = np.linspace(u.min(), u.max(), 1000)
new_points = splev(u_new, tck)

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, y, 'ro')
ax.plot(new_points[0], new_points[1], 'r-')
plt.show()

# Aproximando por poligonos
https://docs.opencv.org/2.4/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html

In [ ]:
def pinta_poligonos(im_contornar,poligonos,lineThickness=1):
    im_contorno = im_contornar.copy()
    cv2.polylines(im_contorno, [poligonos], True, (0, 0, 255), lineThickness)
    return im_contorno

In [ ]:
#transforma o contorno 1 em poligonos
poligonos = cv2.approxPolyDP(con[1],epsilon=0.1,closed = True)
len(poligonos)

In [ ]:
imshow(pinta_poligonos(im,poligonos,1))

### Desenhando contornos em imagem 4 vezes maior

In [ ]:
im_4x = np.ones((400,400,3)).astype('uint8')*255
poligonos_4x = poligonos * 4

In [ ]:
#Sem interpolar

im_pintada = pinta_poligonos(im_4x,poligonos_4x,1)
imshow(im_pintada, True)

In [ ]:
x,y = poligonos_4x.T
x = x[0]
y = y[0]

tck, u = splprep([x, y], s=0, per=1)
u_new = np.linspace(u.min(), u.max(), 1000)
interpolado = np.array(np.around(splev(u_new, tck))).T.astype(int)

In [ ]:
im_pintada = pinta_poligonos(im_4x,interpolado,1)
imshow(im_pintada, True)